In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import saving

In [2]:
import os
from sklearn.model_selection import train_test_split # type: ignore
import shutil

dataset_path = "cancerdataset"
labels = ["Bengin", "Malignant", "Normal"]

data_folder = os.path.join(dataset_path, 'Data')

train_dir = os.path.join(dataset_path, 'train')
val_dir = os.path.join(dataset_path, 'test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

for label in labels:
    os.makedirs(os.path.join(train_dir, label), exist_ok=True)
    os.makedirs(os.path.join(val_dir, label), exist_ok=True)

for label in labels:
    label_folder = os.path.join(data_folder, label)
    images = os.listdir(label_folder)
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)
    
    for image in train_images:
        shutil.copy(os.path.join(label_folder, image), os.path.join(train_dir, label, image))
    
    for image in val_images:
        shutil.copy(os.path.join(label_folder, image), os.path.join(val_dir, label, image))

In [3]:
dataset_path = "cancerdataset"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

training_set = train_datagen.flow_from_directory(
    f'{dataset_path}/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    f'{dataset_path}/test',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

Found 876 images belonging to 3 classes.
Found 221 images belonging to 3 classes.


In [4]:
cnn = tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Flatten())

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))  # Dropout to prevent overfitting
cnn.add(tf.keras.layers.Dense(units=3, activation='softmax'))

cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

cnn.summary()

cnn.fit(x = training_set,  epochs=100)

print(training_set.class_indices)

c:\Users\Matrix Store\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 167,363 (653.76 KB)

 Trainable params: 167,363 (653.76 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100


c:\Users\Matrix Store\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


28/28 ━━━━━━━━━━━━━━━━━━━━ 21s 477ms/step - accuracy: 0.4823 - loss: 0.9878
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 131ms/step - accuracy: 0.5058 - loss: 0.9166
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 6s 154ms/step - accuracy: 0.6313 - loss: 0.8877
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 6s 160ms/step - accuracy: 0.6661 - loss: 0.8077
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 6s 160ms/step - accuracy: 0.7187 - loss: 0.7505
Epoch 6/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 6s 156ms/step - accuracy: 0.7008 - loss: 0.7452
Epoch 7/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 153ms/step - accuracy: 0.6817 - loss: 0.7634
Epoch 8/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 6s 167ms/step - accuracy: 0.7277 - loss: 0.6897
Epoch 9/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 151ms/step - accuracy: 0.7213 - loss: 0.6838
Epoch 10/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 154ms/step - accuracy: 0.7465 - loss: 0.6713
Epoch 11/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 6s 164ms/step - accuracy: 0.7571 - loss: 0.6124
Epoch 12/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 6s 181ms/ste

In [5]:
test_loss, test_acc = cnn.evaluate(x=test_set, verbose=1)

7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 934ms/step - accuracy: 0.9281 - loss: 0.1375


In [8]:
cnn.save('model/Cancer_Model.h5')

In [10]:
# Test the model

import numpy as np
from keras.preprocessing import image
from keras.models import load_model
import os
import random

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

dataset_path = "cancerdataset"
labels = ["Bengin", "Malignant", "Normal"]

test_folders = os.listdir(f'{dataset_path}/test')

random_folder = random.choice(test_folders)
folder_path = os.path.join(f'{dataset_path}/test', random_folder)
images = os.listdir(folder_path)
random_image = random.choice(images)
image_path = os.path.join(folder_path, random_image)

test_image = image.load_img(image_path, target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

cnn = load_model('model/Cancer_Model.h5')

result = cnn.predict(test_image)
prediction = labels[np.argmax(result[0])]

print(image_path)
print(result)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
cancerdataset/test\Bengin\Bengin case (71).jpg
[[1. 0. 0.]]
Bengin
